In [1]:
import json
import torch
import hashlib
import traceback
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "Qwen/Qwen2-7B-Instruct"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device, torch_dtype="auto")
def generate_response(inp, unique_id, max_new_tokens=2048):

    #与训练使用的template保持一致
    try:
        prompt= f"User: {inp}\nAssistant: "
        inputs =tokenizer(prompt, return_tensors="pt").to(model.device) 
        params={
        "max_new_tokens": max_new_tokens,
        "top_p":0.95,
        "temperature": 0.8, 
        "do_sample":True 
        }
        with torch.no_grad():
            outputs =model.generate(**inputs,**params) 
        prompt_leng =len(inputs[0])
        output =outputs[0][prompt_leng:]
        response =tokenizer.decode(output, skip_special_tokens=True) 
        run_info ={
        "unique_id":unique_id,
         "model_inp": prompt, 
         "gen_params": params}
        return response, run_info 
    except Exception as e:
        traceback.print_exc()
        return None,() 



/home/guangyou/.conda/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]


In [2]:
def generate_unique_code(input_string): 
    sha256_hash=hashlib.sha256()
    sha256_hash.update(input_string.encode('utf-8')) 
    unique_code =sha256_hash.hexdigest()
    return unique_code 
def load_test_data(test_file):
    test_data = []
    with open(test_file, 'r', encoding='utf-8') as f:
        for line in f:
            test_data.append(json.loads(line.strip()))
    return test_data

In [3]:
log_file="run_log.jsonl"#日志
ans_file="model_logits.jsonl"#推理结果
  # Load test data
test_data = load_test_data('eval_only_query.jsonl')
print("test")
# prompt_lst=[["简单阐述下当前汽车的发展趋势","subiective"],["为什么中国大力推行新能源汽车","subiective"]] 
for item  in test_data:
    
    prompt=item['query']
    query_type=item['query_type']

    unique_id =generate_unique_code(prompt) 
    # print(prompt)
    #print保留unique_id进行数据追溯
    response, run_info=generate_response(prompt, unique_id) 
    print(response)
    if response is None:
        continue

    run_info["answer"]= response 
    #保存运行日志
    
    with open(log_file,"a") as fw:
        fw.write(json.dumps(run_info, ensure_ascii=False)+"\n") 

    #保存型输结果
    answer_info = {"query": prompt,
    "query_type":query_type, "answer": response}

    with open(ans_file, "a") as fw:
        fw.write(json.dumps(answer_info, ensure_ascii=False)+"\n") 
    print(f"answer_info: {answer_info}")

test
1. 地理信息检索：使用地理信息系统（GIS）进行地理数据的检索。用户可以通过地图上的标记或者输入目的地名称等方式来查找自己想要去的地方。

2. 路线规划：使用GIS软件进行路线规划，提供多种路径选择，如最短路径、最快路径、最少费用路径等，供用户选择。同时，可以为用户提供实时路况信息，以及避开拥堵和施工区域的建议。

3. 实时导航：在路线上提供实时导航功能，帮助用户准确地到达目的地。可以显示当前的位置、行驶的方向、预计到达时间等信息。

4. 景点推荐：根据用户的兴趣爱好和位置信息，推荐附近具有代表性的景点或美食店等，提高旅行体验。

5. 预订服务：集成酒店、餐饮、交通等各种预订服务，为用户提供一站式旅游解决方案。

6. 数据可视化分析：对历史出行数据进行分析，提供个性化行程建议，比如热门景点推荐、旅行高峰期预测等。 

7. 交互式地图：允许用户在地图上做笔记，标记出自己想去的地方，或者是记录已经去过的地方，方便下次查看。 

8. 用户反馈与分享：鼓励用户分享他们的旅行经验和照片，形成一个社区。用户之间可以互相借鉴经验，找到更多的有趣地方。

9. 其他相关服务：例如货币兑换率查询，天气预报，语言翻译等，为旅行提供更多便利。
answer_info: {'query': '如何利用地理数据可视化技术方便用户查找和规划旅行路线？', 'query_type': 'subjective', 'answer': '1. 地理信息检索：使用地理信息系统（GIS）进行地理数据的检索。用户可以通过地图上的标记或者输入目的地名称等方式来查找自己想要去的地方。\n\n2. 路线规划：使用GIS软件进行路线规划，提供多种路径选择，如最短路径、最快路径、最少费用路径等，供用户选择。同时，可以为用户提供实时路况信息，以及避开拥堵和施工区域的建议。\n\n3. 实时导航：在路线上提供实时导航功能，帮助用户准确地到达目的地。可以显示当前的位置、行驶的方向、预计到达时间等信息。\n\n4. 景点推荐：根据用户的兴趣爱好和位置信息，推荐附近具有代表性的景点或美食店等，提高旅行体验。\n\n5. 预订服务：集成酒店、餐饮、交通等各种预订服务，为用户提供一站式旅游解决方案。\n\n6. 数据可视化分析：对历史出行数据进行分析，提供个性化行程建议，比如热门景点推荐、旅行高峰期预测

KeyboardInterrupt: 